# 7-compute-occurence-entities
From the file list_meetings.csv, find the number of invertions of each entitx for each issues. Return a file named occurences.csv with the collected informations

In [1]:
import urllib.request
import requests
import re
from bs4 import BeautifulSoup
from dateutil.parser import parse
import csv
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.tokenize import MWETokenizer
import import_ipynb
from 2-a-extract-paragraphes import clean_page_to_parse
from 2-a-extract-paragraphes import extract_p_tags
from 2-a-extract-paragraphes import extract_p_tags_45

importing Jupyter notebook from extract_p_tags.ipynb


In [2]:
def open_list_meetings():
    f = open('list_meetings.csv')
    return csv.reader(f)

In [3]:
#Extract tuple for each row from Paula's dataset
def extract_tuple(line):
    l = line.replace('"',"")
    l = l.replace('\n',"")
    l = l.split('\t')
    return l

In [4]:
#Clean the sentence by removing special char
def clean_tp(sentence):

    s = sentence.replace("\r\n\s\s+"," ")
    s = s.replace("\r\n"," ")
    s = s.replace("\s\s+"," ")
    s = s.replace("\\."," ")
    s = s.replace("\\r\\n"," ")
    p = re.compile(r'<.*?>')
    return p.sub('', s)


In [5]:
#Extract all the desired sentences (without sponsors)
def extract_from_paragraphes_sentences(list_paragraphes):
    list_paragraphes_foot = set()
    #Remove footer
    for i in range(len(list_paragraphes)):
        if('Sustaining Donors' not in list_paragraphes[i] and 'This issue of' not in list_paragraphes[i]):
            if(list_paragraphes[i] not in list_paragraphes_foot):
                list_paragraphes_foot.add(list_paragraphes[i])
    list_paragraphes_foot= list(list_paragraphes_foot)
    #Split into sentence from paragraph
    splited_sentence = []
    for s in list_paragraphes_foot:
        splited_sentence += sent_tokenize(s)
    return splited_sentence

In [6]:
#Extract complete dataset of Paula, put it into list of tuples
def extract_dataset():
    list_data = list(open('statements_count.csv','r'))
    list_dataset = []
    for line in list_data:
        list_dataset.append(extract_tuple(line))
    return list_dataset


In [7]:
#Extract all the entities mentioned in Paula's dataset
def extract_entities():
    list_dataset = extract_dataset()
    set_entities = set()
    for line in list_dataset:
        set_entities.add(line[0])
    return set_entities

In [8]:
#Count number of time each entity in list_entities is mentioned in list_sentences
def count_occurences(list_sentences, dict_occ, tokenizer, list_entities, meeting):
    list_sentences = list(set(list_sentences))
    for s in list_sentences:
        #Split line into words with tokenizer to detetc entity
        line = s.replace(",","")

        line_splited = word_tokenize(line)
        tokens = tokenizer.tokenize(line_splited)    
        tokens = [clean_tp(token) for token in tokens]

        for entity in list_entities:
            #Increment value of intervention of the entity
            if(entity in tokens):
                dict_occ[entity] += 1                
    rows = [(meeting[0],entity,meeting[4],dict_occ[entity],meeting[5]) for entity in dict_occ]
    return rows

In [9]:
def print_s(list_p):
    outF = open("list_p.txt", "w") 
    for line in list_p:

    # write line to output file
        outF.write(str(line))
        outF.write("\n")

In [10]:
def extract_occurences_issue_ENB():
    #List meetings
    list_ENBs = list(open_list_meetings())[87:91]


    #tokenizer = MWETokenizer(extract_entities(), separator=' ')
    list_entities = list(extract_entities())
    tokens_entities = [l.split(' ') for l in list(extract_entities())]
    tokenizer = MWETokenizer(tokens_entities, separator=' ')
    occurences_meetings = []
    k=0
    #2 Count number time entities mentioned
    for meeting in list_ENBs:
        if(int(meeting[4]) < 45):
            list_paragraphes = [clean_tp(str(line)) for line in extract_p_tags_45(meeting[6])]

        else :
            request = requests.get(meeting[6])
            if(request.status_code == 200):
                list_paragraphes = [clean_tp(str(line)) for line in extract_p_tags(meeting[6])]


                    
        list_sentences = extract_from_paragraphes_sentences(list_paragraphes)
        dict_occurences = dict.fromkeys(extract_entities(), 0)
        occurences_meetings += count_occurences(list_sentences, dict_occurences, tokenizer, list_entities, meeting)

    
    with open('occurences.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        #header
        writer.writerow(('meeting_type','entity','issue_number','interventions','date'))
        writer.writerows(occurences_meetings)






In [11]:
extract_occurences_issue_ENB()


[['Niger'], ['Caribbean', 'Community'], ['Fiji'], ['Benin'], ['Cambodia'], ['Bosnia', 'and', 'Herzegovina'], ['Guinea'], ['LDCs'], ['Monaco'], ['Namibia'], ['Central', 'African', 'Republic'], ['Poland'], ['Belarus'], ['Samoa'], ['Yugoslavia'], ['Slovenia'], ['Uruguay'], ['Dominican', 'Republic'], ['Latvia'], ['Venezuela'], ['Guyana'], ['Uzbekistan'], ['Comoros'], ['ALBA'], ['Maldives'], ['Turkmenistan'], ['Grenada'], ['Costa', 'Rica'], ['Tonga'], ['Paraguay'], ['Niue'], ['Ecuador'], ['Iran'], ['Palau'], ['Ireland'], ['Kenya'], ['Honduras'], ['Vanuatu'], ['EU'], ['Libya'], ['Yemen'], ['Botswana'], ['Angola'], ['Brazil'], ['Suriname'], ['Sweden'], ['Cuba'], ['Lebanon'], ['Central', 'Group'], ['Jordan'], ['AOSIS'], ['Guatemala'], ['Nicaragua'], ['Qatar'], ['SICA'], ['Like', 'Minded', 'Developing', 'Countries'], ['Somalia'], ['Lithuania'], ['Kyrgyz', 'Republic'], ['Burkina', 'Faso'], ['Malawi'], ['Mountain', 'Landlocked', 'Developing', 'Countries'], ['Indonesia'], ['Liberia'], ['United', '

KeyboardInterrupt: 